# Utilisation de transformers pour la classification de textes

 > ℹ️ Inspiré de :
 > - https://github.com/nlp-with-transformers/notebooks/blob/main/02_classification.ipynb
 > - https://huggingface.co/docs/transformers/tasks/sequence_classification

🥅 **Objectifs**

- Savoir utiliser l'écosystème HuggingFace pour réutiliser des modèles pré-entraînés et les affiner sur de nouvelles données

## 1. Installation des librairies nécessaires

In [ ]:
! pip install transformers[torch]

In [ ]:
! pip install datasets

In [ ]:
! pip install evaluate

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
import evaluate
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch

## 2. Récupération et préparation des données

In [ ]:
!mkdir data
!wget -P data https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/winemag-fr_train.csv

In [ ]:
wine_df = pd.read_csv("data/winemag-fr_train.csv", sep=",", dtype={'description': 'object',
                                           'price': 'float64',
                                           'province': 'category',
                                           'variety': 'object'})

In [ ]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(wine_df.province.unique().categories.to_list())
label2id = {class_names[i]:i for i in range(len(class_names))}
id2label = {i:class_names[i] for i in range(len(class_names))}

In [ ]:
data_df = pd.DataFrame()
# Le texte décrivant chaque vin est composé des colonnes variety et description
split_variety = wine_df.variety.str.split('_')
data_df['text'] = split_variety.str.join(' ') + ' ' + wine_df.description
# La classe cible est la région (province) sous forme d'identifiant numérique
data_df['label'] = wine_df.province.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
province_features = Features({'text': Value('string'),
                              'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=province_features)
# Découpage en train et test
data = data.train_test_split(test_size=0.2, shuffle=True, seed=12)

In [ ]:
data['train'].features

In [ ]:
data['train'][0]

## 3. Tokénisation des données

Nous allons utiliser une variante de BERT (pour l'anglais) appelée DistilBERT. Ce modèle obtient des performances comparables à BERT, mais est de plus petite taille et plus rapide.

🚨 **DistilBERT est un modèle pour l'anglais et ne doit donc pas être utilisé pour des textes dans une autre langue. Pour rechercher des modèles adaptés à une autre langue, utiliser le filtre "Languages" sur https://huggingface.co/models** 🚨

In [ ]:
model_ckpt = "distilbert-base-uncased"
# Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
# Tokénisation des 2 premières instances
preprocess_function(data['train'][:2])

- Les 0 à la fin sont le résultat du padding (toutes les séquences du lot ont la même longueur)
- Les 0 dans le masque d'attention indiquent les tokens à ignorer dans le mécanisme d'attention (tokens ajoutés par le padding)

In [ ]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=None)

In [ ]:
tokenized_data['train'][0]

In [ ]:
# Affichage des tokens. DistilBERT utilise l'algorithme WordPiece
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['train'][0]['input_ids'])
print(tokenized_data['train'][0]['text'])
print(tokens)

- [CLS] et [SEP] indiquent le début et la fin de la séquence.
- Les tokens sont en minuscules.
- Le préfixe ## indique que le sous-mot n'est pas précédé par une espace

In [ ]:
# Taille du vocabulaire
tokenizer.vocab_size

In [ ]:
# Taille de contexte maximum
tokenizer.model_max_length

## 4. Préparation de l'évaluation

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"accuracy": acc['accuracy'], "f1-macro": f1["f1"]}

## 5. Entraînement par affinage

On commence par charger le modèle pré-entraîné

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size = 64
training_args = TrainingArguments(
    output_dir=f"{model_ckpt}-finetuned-wine",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def init_trainer():
  model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=len(class_names), id2label=id2label, label2id=label2id
    ).to(device)
  return Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_data["train"],
      eval_dataset=tokenized_data["test"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  ), model

In [ ]:
trainer, model = init_trainer()
trainer.train()

❓ [1] Que constatez-vous par rapport aux résultats obtenus précédemment pour ce jeu de données ? Attention, ici nous ne faisons pas de validation croisée à 5 plis, les résultats sont évalués uniquement sur 20% des données (un seul pli).

In [ ]:
trainer2, model2 = init_trainer()
trainer2.train()

❓ [2] Que constatez-vous par rapport aux résultats d'entraînement obtenus pour la cellule précédente ? Est-ce que les résultats sont les mêmes ?

## 6. Analyse des résultats

In [ ]:
# Prédictions pour les données de test
preds_output = trainer.predict(tokenized_data['test'])

In [ ]:
preds_output

In [ ]:
preds_output.metrics

---
Nous allons également afficher la matrice de confusion.





In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
y_valid = tokenized_data['test']['label']

In [ ]:
labels = tokenized_data['test'].features['label'].names

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=labels_for_fig)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

plot_confusion_matrix(y_preds, y_valid, labels)

❓ [3] Que constatez-vous ? Quelle est la classe pour laquelle les résultats sont les moins bons ? Pourquoi ?

Enfin, nous allons analyser les erreurs de classification. Pour cela, nous allons trier les instances par perte décroissante.

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Fonction qui retourne la perte (entropie croisée) et la classe prédite
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                             reduction="none")
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
# Conversion des données au bon format
tokenized_data.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Calcul des valeurs de perte
tokenized_data["test"] = tokenized_data["test"].map(
    forward_pass_with_label, batched=True, batch_size=64)

In [ ]:
# Création d'un DataFrame avec les textes, les pertes les classe (prédites et attendues)

def label_int2str(row):
    return tokenized_data["train"].features["label"].int2str(row)

tokenized_data.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = tokenized_data["test"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [ ]:
# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1)
# Affichage des 10 premières instances triées par perte décroissante
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
# Affichage des 10 premières instances triées par perte croissante
# Cela permet de voir les instances pour lesquelles les prédictions sont les plus certaines
df_test.sort_values("loss", ascending=True).head(10)